In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('NIJ_s_Recidivism_Chall.csv')

In [4]:
print(df)

          ID Gender   Race Age_at_Release  Residence_PUMA Gang_Affiliated  \
0          1      M  BLACK          43-47              16           False   
1          2      M  BLACK          33-37              16           False   
2          3      M  BLACK    48 or older              24           False   
3          4      M  WHITE          38-42              16           False   
4          5      M  WHITE          33-37              16           False   
...      ...    ...    ...            ...             ...             ...   
25830  26756      M  BLACK          23-27               9           False   
25831  26758      M  WHITE          38-42              25           False   
25832  26759      M  BLACK          33-37              15           False   
25833  26760      F  WHITE          33-37              15             NaN   
25834  26761      M  WHITE          28-32              12           False   

       Supervision_Risk_Score_First Supervision_Level_First  \
0           